In [17]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
from dash import ctx, callback
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os


# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


import animalShelter
from animalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# Username and password for database
username = "aacuser"
password = "AACPass"
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

image_filename = 'GraziosoSalvareLogo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Center(html.B(html.H1('Project Two - Jeremy Depenhart'))),
    html.Hr(),
    html.Div(className='buttonRow',
            style={'display' : 'flex'},
                children=[
                html.Button(id='submit-button-one', n_clicks=0, children='Water'),
                html.Button(id='submit-button-two', n_clicks=0, children='Mountain/Wilderness'),
                html.Button(id='submit-button-three', n_clicks=0, children='Disaster/Individual Tracking'),
                html.Button(id='submit-button-four', n_clicks=0, children='Reset')
                ]),
    dash_table.DataTable(id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} 
                for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current= 0,
        page_size= 10,               
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
# Buttons
@app.callback(
    Output('datatable-id', "data"),
    [Input('submit-button-one', 'n_clicks'),
     Input('submit-button-two', 'n_clicks'),
     Input('submit-button-three', 'n_clicks'),
     Input('submit-button-four', 'n_clicks')
    ])
def on_click(button1, button2, button3, button4):
    # start case
    df = pd.DataFrame.from_records(db.read({}))
    
    # Handles button input
    if (ctx.triggered_id == 'submit-button-one'):
        df = pd.DataFrame.from_records(db.read({"animal_type": "Dog",
                "breed": {"$in": ["Labrador Retriever Mix","Chesapeake Bay Retriever", "Newfoundland"]},
                "sex_upon_outcome": "Intact Female",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}}))                                         
    elif (ctx.triggered_id == 'submit-button-two'):
        df = pd.DataFrame.from_records(db.read({"animal_type": "Dog",
                "breed": {"$in": ["German Shepherd","Alaskan Malamute", "Old English Sheepdog", 
                                  "Siberian Husky", "Rottweiler"]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}}))  
    elif (ctx.triggered_id == 'submit-button-three'):
        df = pd.DataFrame.from_records(db.read({"animal_type": "Dog",
                "breed": {"$in": ["Doberman Pinscher","German Shepherd", "Golden Retriever", 
                                  "Bloodhound", "Rottweiler"]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":20.0, "$lte":300.0}}))  
    elif (ctx.triggered_id == 'submit-button-four'):
        df = pd.DataFrame.from_records(db.read({}))
    # Cleanup Mongo _id feild
    df.drop(columns=['_id'], inplace=True)
    return df.to_dict('records')

# Chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    dff2 = pd.DataFrame.from_dict(viewData)
    figure = px.pie(dff2, names='breed', title='Preferred Animals')
    figure.update_traces(textposition='inside')
    figure.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
    
    return [dcc.Graph(figure=figure)]

# Map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    dff = pd.DataFrame.from_dict(viewData)
    if index is None:
        row = 0
    else:
        row = index[0]
           
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
           center=[30.75,-97.48], zoom=9, children=[
           dl.TileLayer(id="base-layer-id"),
           # Marker with tool tip and popup
           # Column 13 and 14 define the grid-coordinates for 
           # the map
           # Column 4 defines the breed for the animal
           # Column 9 defines the name of the animal
           dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]],
              children=[
              dl.Tooltip(dff.iloc[row,4]),
              dl.Popup([
                 html.H1("Animal Name"),
                html.P(dff.iloc[row,9])
             ])
          ])
       ])
    ]

app.run_server(debug=True)    

Dash app running on http://127.0.0.1:20810/
